In [50]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import emoji
import nltk
from tqdm import tqdm
from collections import Counter
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag, ngrams
from nltk.stem import WordNetLemmatizer

tqdm.pandas()

In [51]:
## All data
data = pd.read_csv("TopGamesData.csv", usecols=["content", "score", "year", "game_name"])

## Among Us data
# data = data[data['game_name']=='AmongUs']

## 10000 Among Us data
# data = (data[data['game_name']=='AmongUs']
#       .groupby('score', group_keys=False)
#       .apply(lambda x: x.sample(n=2000, random_state=42) if len(x) > 2000 else x, include_groups=False)
#       .reset_index(drop=True)
# )
data

,game_name,content,score,year
0,8 Ball Pool,After having an account that I spent actual mo...,2,2025
1,8 Ball Pool,"Fun game, but has WAY too many ads to try to t...",1,2025
2,8 Ball Pool,I quit playing this for several years despite ...,2,2025
3,8 Ball Pool,I love this game but it is too easy to select ...,3,2025
4,8 Ball Pool,"The game itself is fun and well-designed, but ...",1,2025
...,...,...,...,...
2034512,Shadow Fight 2,Shadow fight is very interested game,5,2022
2034513,Shadow Fight 2,best game ever played in the world it is very ...,5,2019
2034514,Shadow Fight 2,My favourite game since 2017 but l left this b...,5,2023
2034515,Shadow Fight 2,I like shadow fight 2 because it remains me of...,5,2022


In [ ]:
data["length"] = data["content"].astype(str).str.len()
data = data[data["length"] <= 500]
data = data[data["score"] != 0]
data

,game_name,content,score,year,length
0,8 Ball Pool,After having an account that I spent actual mo...,2,2025,489
1,8 Ball Pool,"Fun game, but has WAY too many ads to try to t...",1,2025,485
2,8 Ball Pool,I quit playing this for several years despite ...,2,2025,443
3,8 Ball Pool,I love this game but it is too easy to select ...,3,2025,494
4,8 Ball Pool,"The game itself is fun and well-designed, but ...",1,2025,369
...,...,...,...,...,...
2034512,Shadow Fight 2,Shadow fight is very interested game,5,2022,36
2034513,Shadow Fight 2,best game ever played in the world it is very ...,5,2019,66
2034514,Shadow Fight 2,My favourite game since 2017 but l left this b...,5,2023,126
2034515,Shadow Fight 2,I like shadow fight 2 because it remains me of...,5,2022,60


In [54]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons 😀😅😂🤣😇🙃🙂
    "\U0001F300-\U0001F5FF"  # symbols & pictographs 🌍🌈⛄
    "\U0001F680-\U0001F6FF"  # transport & map symbols 🚀🚗🛳️
    "\U0001F1E0-\U0001F1FF"  # flags 🇮🇩🇺🇸
    "\U00002702-\U000027B0"  # dingbats ✂️✈️✔️
    "\U000024C2-\U0001F251"  # enclosed characters 🅰️🆗
    "]+",
    flags=re.UNICODE,
)

has_emoji = data["content"].astype(str).apply(lambda x: bool(emoji_pattern.search(x)))

print("Number of reviews containing emoji:", has_emoji.sum())
print("Percentage of reviews with emoji:", round(has_emoji.mean() * 100, 2), "%")

Number of reviews containing emoji: 197733
Percentage of reviews with emoji: 9.74 %


In [55]:
StopWords = set(stopwords.words("english"))

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # fallback
    
Lemmatizer = WordNetLemmatizer()

In [ ]:
def clean(review):
    # lower text
    review = review.lower()

    # translate emoji
    review = emoji.demojize(review)
    review = review.replace("_", " ")
    review = re.sub(r":([a-z0-9 ]+):", r"\1", review)

    #remove non-alphabet and non-numbers 
    review = re.sub(r'[^a-z0-9\s]+', '', review)    

    #remove stopwords
    words = [w for w in review.split() if w not in StopWords]

    #lemmatize words
    tagged_words = pos_tag(words)
    lemmatized = [Lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in tagged_words]
    return " ".join(lemmatized)

if os.path.exists("TopGamesDataClean.csv"):
    data = pd.read_csv("TopGamesDataClean.csv")
else:
    data['content'] = data['content'].progress_apply(clean)
    data = data.dropna()
    data.to_csv("TopGamesDataClean.csv", index=False)
data

,game_name,content,score,year
0,8 Ball Pool,account spend actual money ban exploit guess m...,2,2025
1,8 Ball Pool,fun game way many ad try take cash point somet...,1,2025
2,8 Ball Pool,quit play several year despite liking game cra...,2,2025
3,8 Ball Pool,love game easy select level may want drain coi...,3,2025
4,8 Ball Pool,game fun welldesigned amount advertisement rea...,1,2025
...,...,...,...,...
2029346,Shadow Fight 2,shadow fight interested game,5,2022
2029347,Shadow Fight 2,best game ever play world best offline game,5,2019
2029348,Shadow Fight 2,favourite game since 2017 l leave become dada ...,5,2023
2029349,Shadow Fight 2,like shadow fight 2 remain child hood,5,2022
